<a href="https://colab.research.google.com/github/clduab11/judicAIta/blob/main/examples/notebooks/train_tunix_reasoning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Judicaita: GRPO Training with Google Tunix on TPU

This notebook demonstrates **GRPO (Group Relative Policy Optimization)** training for the Judicaita legal AI assistant using:-

- **Google Tunix** for RL training infrastructure-
- **Gemma 3-1B-IT** as the base model-
- **TPU v2-8+** for accelerated training-
- **LoRA adapters** for parameter-efficient fine-tuning

This is developed for the Kaggle hackathon to train models that generate explainable legal reasoning with structured XML-formatted outputs.

## ⚡ TPU Requirements

**IMPORTANT**: This notebook requires:-
- Google Colab with TPU runtime (TPU v2-8 or higher)-
- Runtime type: TPU (not CPU or GPU)-
- To enable: Runtime → Change runtime type → Hardware accelerator: TPU

## 📋 What This Notebook Does

1. **Environment Setup + TPU Init (Combined)**: Install Tunix and dependencies, initialize TPU - **NO RESTART NEEDED**
2. **HuggingFace Authentication**: Login to download Gemma models
3. **Model Loading**: Download and initialize Gemma 3-1B-IT with LoRA
4. **Dataset Preparation**: Format training data with XML-tagged reasoning
5. **Reward Function**: Multi-objective scoring including **Legal Accuracy**, **Reasoning Coherence**, **Answer Correctness** (35%), Format, and Length.
6. **GRPO Training**: Train with `GRPOLearner` and `RLCluster` on TPU
7. **Export**: Package trained LoRA adapters for Kaggle submission

## 🔄 Data Flow

```
Dataset → Prompts → Model Rollouts → Reward Scoring → GRPO Updates
                    ↓
         LoRA Adapter Checkpoints
```

## ⚠️ Differences from Main Codebase

| Aspect | Main Codebase | This Notebook |
|--------|---------------|---------------|
| Format | Step-by-step format | XML `<reasoning>`/`<answer>` |
| Framework | PyTorch | JAX/Flax |
| Training | Custom GRPO | Tunix GRPOLearner |
| Hardware | GPU/CPU | TPU v2-8+ |

## ✅ Recent Changes (Jan 2025)

**Fixed: JAX/TPU SIGSEGV on Step 2 initialization**-

- ✅ Combined Step 1 (dependencies) and Step 2 (TPU init) into single cell-
- ✅ No more mid-notebook kernel restart required-
- ✅ Uses Colab's pre-installed JAX (no version conflicts)-
- ✅ Pins `google-tunix==0.1.6` for stability-
- ✅ Guards against redundant installs-
- ✅ Immediate TPU smoke test

**This fixes the SIGSEGV crash that occurred when restarting the kernel between dependency installation and TPU initialization.**

## 📚 References

- [Google Tunix Documentation](https://tunix.readthedocs.io/)-
- [Tunix GRPO Gemma Example](https://github.com/google/tunix/tree/main/examples/grpo_gemma)-
- [Gemma Model Card](https://ai.google.dev/gemma/docs)-
- [GRPO Paper](https://arxiv.org/abs/2402.03300)-
- [Judicaita Repository](https://github.com/clduab11/judicAIta)

## ⚠️ Known Limitations

- **TPU Required**: Cannot run on CPU/GPU without code modifications-
- **Memory**: TPU v2-8 has ~64GB; larger models may need v3 or higher-
- **Dataset**: Assumes generic legal reasoning tasks (not LegalBench-specific)-
- **Checkpoints**: Large checkpoint files may exceed Colab storage limits-
- **API Stability**: Tunix API may change; verify imports match your version

## 🎯🚀 Step(s) 1+2 , Task 1 - IMPORTANT!: Dependencies + TPU Init (NO RESTART)

**IMPORTANT**: This cell combines dependency installation and TPU initialization to eliminate the mid-notebook restart issue that causes SIGSEGV crashes.

### What this cell does:

1. Removes RAPIDS cruft that conflicts with our stack
2. Checks if core dependencies are already installed (skip if present)
3. Installs only what's needed:
   - `google-tunix==0.1.6` (pinned version)
      - `transformers`, `datasets`, `wandb`, `flax` (compatible versions)
         - **Does NOT override JAX** - uses Colab's pre-installed JAX
         4. Initializes TPU runtime immediately (no restart needed)
         5. Runs smoke test to verify TPU is working

         ### Key differences from old Step 1+2:

         - ❌ **No more kernel restart between steps**
         - ✅ Uses Colab's pre-installed JAX (no version conflicts)
         - ✅ Pins `google-tunix==0.1.6` (not bleeding edge 0.5.0+)
         - ✅ Guards against redundant installs
         - ✅ Immediate TPU verification

         **Expected output:**
         - ✅ Core dependencies present or installed
         - ✅ TPU devices detected (8 cores for TPU v3-8)
         - ✅ Smoke test passed (matmul on TPU)

In [ ]:
# ============================================================
# Step 1+2 Combined: Dependencies + TPU Init (NO RESTART)
# ============================================================
import sys
import subprocess
import os

# Suppress TF warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# 🧹 Remove RAPIDS cruft that conflicts with our stack
print("🧹 Cleaning up conflicting packages...")
try:
    subprocess.run(
        [sys.executable, "-m", "pip", "uninstall", "-y", "cuml-cu12", "cudf-cu12"],
        capture_output=True,
        check=False,
    )
    print("✅ Cleanup complete")
except Exception as e:
    print(f"⚠️  Cleanup warning (non-critical): {e}")

# Check if we need to install anything
try:
    import tunix
    import transformers
    import datasets
    import flax

    print("\n✅ Core dependencies already present, skipping install...")
    print(
        f"   Tunix version: {tunix.__version__ if hasattr(tunix, '__version__') else 'unknown'}"
    )
    print(f"   Transformers version: {transformers.__version__}")
    print(f"   Flax version: {flax.__version__}")
    skip_install = True
except ImportError:
    print("\n📦 Installing dependencies (don't touch JAX)...")
    skip_install = False

if not skip_install:
    # Install core dependencies - DON'T override JAX
    subprocess.check_call(
        [
            sys.executable,
            "-m",
            "pip",
            "install",
            "-q",
            "google-tunix==0.1.6",  # Pinned version
            "transformers>=4.40.0,<4.57.1",
            "datasets",
            "wandb",
            "flax>=0.10.2,<0.13.0",  # Compatible range
        ]
    )
    print("✅ Installed. Continuing WITHOUT restart...")

# ============================================================
# TPU Initialization - use Colab's pre-installed JAX
# ============================================================
print("\n🚀 Initializing TPU runtime...")
import jax
import jax.numpy as jnp

print(f"\n🔧 JAX version: {jax.__version__}")
print(f"📍 Backend: {jax.default_backend()}")

# Get TPU devices
devices = jax.devices()
print(f"\n🎯 TPU devices: {len(devices)}")
for i, d in enumerate(devices):
    print(f"   [{i}] {d}")

if len(devices) == 0:
    raise RuntimeError(
        "❌ No TPU devices detected! Please set runtime to TPU: Runtime → Change runtime type → TPU"
    )

# ============================================================
# Smoke test - verify TPU is working
# ============================================================
print("\n🧪 Running TPU smoke test...")
try:
    x = jnp.ones((1000, 1000))
    y = jnp.dot(x, x)
    print(f"✅ TPU smoke test passed!")
    print(f"   Matmul result shape: {y.shape}")
    print(f"   Sample value: {y[0, 0]}")
except Exception as e:
    print(f"❌ TPU smoke test failed: {e}")
    raise
print("\n" + "=" * 60)
print("🎉 SUCCESS: Combined Step 1+2 complete!")
print("=" * 60)
print("✅ Dependencies installed")
print("✅ TPU initialized and verified")
print("✅ No restart needed")
print("\nYou can now proceed to Step 3 (HuggingFace authentication)")


## 🔐 Step 3: Authenticate with Hugging Face

Login to Hugging Face to download the Gemma model.

In [ ]:
from huggingface_hub import login, snapshot_download
import os

# Login to Hugging Face
# You'll be prompted to enter your HF token
# Get your token from: https://huggingface.co/settings/tokens
print("Please enter your Hugging Face token:")
login()

print("\n✅ Authenticated with Hugging Face!")

## 📥 Step 4: Download Gemma 3-1B-IT Model

Download the model files and initialize the tokenizer.

**Note**: Using `gemma-3-1b-it` as it's the latest available Gemma instruction-tuned model. Update to `gemma-3-1b-it` if/when available.

In [ ]:
from transformers import AutoTokenizer
import os

# Download model
MODEL_ID = "google/gemma-3-1b-it"  # Using gemma-3-1b-it as gemma-3-1b-it may not be available yet
CACHE_DIR = "./gemma_model_cache"

print(f"Downloading {MODEL_ID}...")
model_path = snapshot_download(
    repo_id=MODEL_ID,
    cache_dir=CACHE_DIR,
    local_dir=f"{CACHE_DIR}/gemma",
    local_dir_use_symlinks=False
)
print(f"✅ Model downloaded to: {model_path}")

# Initialize tokenizer
print("\nInitializing tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_path)
print(f"✅ Tokenizer initialized")
print(f"   Vocab size: {len(tokenizer)}")
print(f"   Special tokens: {tokenizer.special_tokens_map}")

# Test tokenization
test_text = "What is the legal precedent for breach of contract?"
tokens = tokenizer(test_text, return_tensors="np")
print(f"\n📝 Test tokenization:")
print(f"   Input: {test_text}")
print(f"   Token count: {len(tokens['input_ids'][0])}")

## 🔧 Step 5: Create Preprocessing Function

Gemma models don't have native system role support. We'll prepend the system prompt to the first user turn.

In [ ]:
def preprocess_with_system_prompt(messages, system_prompt):
    """
    Prepend system prompt to first user message.

    Gemma doesn't support system role natively, so we merge it with
    the first user turn as a workaround.

    Args:
        messages: List of message dicts with 'role' and 'content'
        system_prompt: System instruction string

    Returns:
        Modified messages list with system prompt prepended
    """
    if not messages:
        return messages

    processed = messages.copy()

    # Find first user message
    for i, msg in enumerate(processed):
        if msg.get('role') == 'user':
            # Prepend system prompt
            original_content = msg['content']
            processed[i]['content'] = f"{system_prompt}\n\n{original_content}"
            break

    return processed

# Define system prompt for legal reasoning
SYSTEM_PROMPT = """You are a legal AI assistant. For each question, provide your analysis in this exact format:
<reasoning>Your step-by-step legal reasoning here. Include relevant legal principles, precedents, and analysis. Aim for at least 100 tokens of detailed reasoning.</reasoning>
<answer>Your final answer or conclusion here.</answer>

Always use this XML format and ensure your reasoning is thorough and well-explained."""

# Test preprocessing
test_messages = [
    {"role": "user", "content": "Is a non-compete clause enforceable in California?"}
]
processed = preprocess_with_system_prompt(test_messages, SYSTEM_PROMPT)
print("📝 Test preprocessing:")
print(f"Original: {test_messages[0]['content'][:50]}...")
print(f"\nProcessed length: {len(processed[0]['content'])} chars")
print(f"System prompt prepended: {'<reasoning>' in processed[0]['content']}")
print("\n✅ Preprocessing function ready!")

## 📊 Task 2: Prepare Training Dataset

Create a dataset with XML-tagged reasoning format compatible with Tunix GRPO.

### JSONL Format Requirements

Each training example must be a JSON object with:
- `prompt`: The question or task
- `ground_truth`: The correct answer for evaluation
- `metadata` (optional): Additional info like task_id, difficulty, etc.

In [ ]:
import json
import re
from typing import List, Dict, Any
from datasets import load_dataset

def prepare_dataset_for_tunix(examples: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Prepare dataset in Tunix-compatible JSONL format.

    Args:
        examples: List of dicts with 'question' and 'answer' fields

    Returns:
        List of dicts with 'prompt', 'ground_truth', and 'metadata'
    """
    prepared = []

    for idx, ex in enumerate(examples):
        prepared.append({
            "prompt": ex.get("question", ex.get("prompt", "")),
            "ground_truth": ex.get("answer", ex.get("ground_truth", "")),
            "metadata": {
                "example_id": idx,
                "original_question": ex.get("question", ""),
                "task_type": ex.get("task_type", "general_reasoning")
            }
        })

    return prepared

print("📥 Loading real legal data from HuggingFace (nguha/legalbench)...")

# Load subset: 'contract_qa' (Contract Law questions)
try:
    dataset = load_dataset("nguha/legalbench", "contract_qa", split="train")
    print(f"   Loaded {len(dataset)} examples from LegalBench (contract_qa)")

    # Take first 100 examples for this demo/hackathon training
    # (In full training, use more)
    real_examples = []
    for item in dataset.select(range(min(len(dataset), 100))):
        real_examples.append({
            "question": item["question"],
            "answer": item["answer"], # LegalBench uses 'answer' usually yes/no or text
            "task_type": "contract_qa"
        })

except Exception as e:
    print(f"⚠️ Failed to load LegalBench: {e}")
    print("   Falling back to synthetic examples for demonstration.")
    real_examples = [
        {
            "question": "Can an employer in California enforce a non-compete clause against a former employee?",
            "answer": "No, non-compete clauses are generally unenforceable in California except in limited circumstances involving sale of business or dissolution of partnership.",
            "task_type": "legal_qa"
        },
        {
            "question": "What is the statute of limitations for filing a breach of contract claim?",
            "answer": "The statute of limitations varies by jurisdiction. In many states, it is 4-6 years for written contracts and 2-3 years for oral contracts.",
            "task_type": "legal_qa"
        },
        {
            "question": "Under what circumstances can a contract be voided for duress?",
            "answer": "A contract can be voided for duress when one party was forced to enter the agreement through threats, violence, or other improper pressure that overcame their free will.",
            "task_type": "legal_qa"
        },
        {
            "question": "What is required to establish an attorney-client privilege?",
            "answer": "Attorney-client privilege requires: (1) an attorney-client relationship, (2) confidential communication, (3) made for the purpose of seeking or providing legal advice.",
            "task_type": "legal_qa"
        },
    ]

# Prepare dataset
prepared_dataset = prepare_dataset_for_tunix(real_examples)

print(f"✅ Prepared {len(prepared_dataset)} training examples")
print(f"\n📝 Sample example:")
print(json.dumps(prepared_dataset[0], indent=2))

### Prompt Template with XML Format

Create a template that formats prompts to expect XML-tagged reasoning.

In [ ]:
def create_prompt_template(question: str, system_prompt: str = SYSTEM_PROMPT) -> str:
    """
    Create a formatted prompt with XML output expectations.

    Args:
        question: The legal question to answer
        system_prompt: System instructions for format

    Returns:
        Formatted prompt string
    """
    template = f"""{system_prompt}

Question: {question}

Response:"""
    return template

def validate_xml_format(response: str) -> bool:
    """
    Validate that response contains proper XML tags.

    Args:
        response: Model generated response

    Returns:
        True if valid XML format, False otherwise
    """
    # Check for both opening and closing tags
    has_reasoning = '<reasoning>' in response and '</reasoning>' in response
    has_answer = '<answer>' in response and '</answer>' in response

    return has_reasoning and has_answer

# Apply template to all examples
templated_prompts = []
for example in prepared_dataset:
    templated = {
        "prompt": create_prompt_template(example["prompt"]),
        "ground_truth": example["ground_truth"],
        "metadata": example["metadata"],
        "original_prompt": example["prompt"]
    }
    templated_prompts.append(templated)

print(f"✅ Created {len(templated_prompts)} templated prompts")
print(f"\n📝 Sample templated prompt (first 300 chars):")
print(templated_prompts[0]["prompt"][:300])
print("...")

# Test validation
test_valid = "<reasoning>This is reasoning</reasoning><answer>This is answer</answer>"
test_invalid = "This is just text without tags"
print(f"\n✅ Validation test:")
print(f"   Valid format: {validate_xml_format(test_valid)}")
print(f"   Invalid format: {validate_xml_format(test_invalid)}")

### Tokenization and Batching

Tokenize prompts and prepare batches for training.

In [ ]:
import numpy as np
from typing import List, Dict

# Set maximum prompt length
MAX_PROMPT_LENGTH = 512  # Adjust based on your needs (512 or 1024)
MAX_RESPONSE_LENGTH = 512

def tokenize_prompts(prompts: List[str], tokenizer, max_length: int = MAX_PROMPT_LENGTH):
    """
    Tokenize prompts with padding and truncation.

    Args:
        prompts: List of prompt strings
        tokenizer: HuggingFace tokenizer
        max_length: Maximum token length

    Returns:
        Dict with input_ids and attention_mask
    """
    tokenized = tokenizer(
        prompts,
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="np"
    )
    return tokenized

def create_training_batches(dataset: List[Dict], batch_size: int = 4):
    """
    Create batches from dataset.

    Args:
        dataset: List of training examples
        batch_size: Number of examples per batch

    Returns:
        List of batches, each batch is a list of examples
    """
    batches = []
    for i in range(0, len(dataset), batch_size):
        batch = dataset[i:i + batch_size]
        batches.append(batch)
    return batches

# Tokenize all prompts
all_prompts = [ex["prompt"] for ex in templated_prompts]
tokenized_prompts = tokenize_prompts(all_prompts, tokenizer, MAX_PROMPT_LENGTH)

print(f"✅ Tokenized {len(all_prompts)} prompts")
print(f"   Max length: {MAX_PROMPT_LENGTH} tokens")
print(f"   Shape: {tokenized_prompts['input_ids'].shape}")

# Create final dataset for training
training_dataset = []
for i, ex in enumerate(templated_prompts):
    training_dataset.append({
        "prompt": ex["prompt"],
        "prompt_tokens": tokenized_prompts['input_ids'][i],
        "attention_mask": tokenized_prompts['attention_mask'][i],
        "ground_truth": ex["ground_truth"],
        "metadata": ex["metadata"]
    })

print(f"\n✅ Final training dataset: {len(training_dataset)} examples")
print(f"   Each example has: {list(training_dataset[0].keys())}")

# Validate dataset format
required_fields = ["prompt", "ground_truth", "metadata"]
all_valid = all(all(field in ex for field in required_fields) for ex in training_dataset)
print(f"\n✅ Dataset validation: {'PASSED' if all_valid else 'FAILED'}")

if not all_valid:
    print("❌ Some examples missing required fields!")
else:
    print("   All examples have required fields: prompt, ground_truth, metadata")

## 🎯 Task 3: Implement Custom Reward Function

Create a competition-compliant reward function that scores:
1. **Answer Correctness** (35%): Match with ground truth (exact or Jaccard)
2. **Legal Accuracy** (25%): Valid legal citation patterns (e.g., U.S.C., v., §)
3. **Reasoning Coherence** (25%): Structural integrity and lack of repetition
4. **Format Compliance** (10%): Proper XML `<reasoning>` and `<answer>` tags
5. **Reasoning Length** (5%): Encouraging detailed analysis (>150 tokens)


In [ ]:
import re
from typing import Tuple, Optional

def extract_xml_content(response: str) -> Tuple[Optional[str], Optional[str]]:
    """
    Extract content from <reasoning> and <answer> XML tags.

    Args:
        response: Model-generated response string

    Returns:
        Tuple of (reasoning_content, answer_content)
        Returns (None, None) if tags are malformed or missing
    """
    try:
        # Extract reasoning
        reasoning_match = re.search(r'<reasoning>(.*?)</reasoning>', response, re.DOTALL)
        reasoning = reasoning_match.group(1).strip() if reasoning_match else None

        # Extract answer
        answer_match = re.search(r'<answer>(.*?)</answer>', response, re.DOTALL)
        answer = answer_match.group(1).strip() if answer_match else None

        return reasoning, answer
    except Exception as e:
        print(f"Warning: Error extracting XML content: {e}")
        return None, None

# Test extraction with edge cases
test_cases = [
    # Valid case
    "<reasoning>Step by step analysis here</reasoning><answer>Final answer</answer>",
    # Missing tags
    "Just plain text without tags",
    # Partial tags
    "<reasoning>Incomplete reasoning",
    # Nested content
    "<reasoning>Analysis with <term>nested</term> content</reasoning><answer>Yes</answer>",
    # Multi-line
    """<reasoning>
Line 1 of reasoning
Line 2 of reasoning
</reasoning>
<answer>Final answer</answer>"""
]

print("🧪 Testing XML extraction:")
for i, test in enumerate(test_cases, 1):
    reasoning, answer = extract_xml_content(test)
    print(f"\nTest {i}:")
    print(f"  Reasoning found: {reasoning is not None}")
    print(f"  Answer found: {answer is not None}")
    if reasoning:
        print(f"  Reasoning preview: {reasoning[:50]}...")
    if answer:
        print(f"  Answer: {answer}")

print("\n✅ XML extraction function tested with edge cases")

In [ ]:
import re
from typing import Tuple, Optional, List, Dict

def extract_xml_content(response: str) -> Tuple[Optional[str], Optional[str]]:
    """
    Extract content from <reasoning> and <answer> XML tags.

    Args:
        response: Model-generated response string

    Returns:
        Tuple of (reasoning_content, answer_content)
        Returns (None, None) if tags are malformed or missing
    """
    try:
        # Extract reasoning
        reasoning_match = re.search(r'<reasoning>(.*?)</reasoning>', response, re.DOTALL)
        reasoning = reasoning_match.group(1).strip() if reasoning_match else None

        # Extract answer
        answer_match = re.search(r'<answer>(.*?)</answer>', response, re.DOTALL)
        answer = answer_match.group(1).strip() if answer_match else None

        return reasoning, answer
    except Exception as e:
        print(f"Warning: Error extracting XML content: {e}")
        return None, None

def compute_format_reward(response: str) -> float:
    """
    Reward for valid XML format (10% weight).
    """
    reasoning, answer = extract_xml_content(response)

    # Check both tags present and have content
    if reasoning is not None and answer is not None:
        if len(reasoning.strip()) > 0 and len(answer.strip()) > 0:
            return 1.0

    return 0.0

def compute_legal_accuracy_reward(response: str, query_context: str = "") -> float:
    """
    Reward for using proper legal citation format (25% weight).
    Checks for presence of standard legal citation patterns.
    """
    reasoning, _ = extract_xml_content(response)
    if not reasoning:
        return 0.0

    # Basic legal citation patterns
    patterns = [
        r'\d+\s+U\.S\.C\.',       # US Code (e.g., 17 U.S.C.)
        r'v\.',                   # Case names (Plaintiff v. Defendant)
        r'§',                     # Section symbol
        r'Article\s+[IVX]+',      # Articles
        r'See\s+also',            # Legal writing style
        r'Id\.',                  # Citation shorthand
        r'Cir\.',                 # Circuit courts
        r'Cal\.',                 # California codes (example)
        r'Rev\.',                 # Review
    ]

    matches = 0
    for pattern in patterns:
        if re.search(pattern, reasoning, re.IGNORECASE):
            matches += 1

    # Cap at 1.0
    return min(1.0, max(0.2, matches * 0.5) if matches > 0 else 0.0)

def compute_reasoning_coherence_reward(response: str) -> float:
    """
    Reward for coherence (25% weight).
    Penalizes repetition and rewards structure.
    """
    reasoning, _ = extract_xml_content(response)
    if not reasoning:
        return 0.0

    # 1. Repetition penalty
    sentences = [s.strip() for s in reasoning.split('.') if len(s.strip()) > 10]
    if not sentences:
        return 0.0

    unique_sentences = set(sentences)
    repetition_ratio = len(unique_sentences) / len(sentences)

    # 2. Structure heuristic
    has_paragraphs = '\n\n' in reasoning
    transitions = ['Therefore', 'However', 'Furthermore', 'Accordingly', 'Thus']
    has_transitions = any(t in reasoning for t in transitions)

    # Combine
    score = repetition_ratio * 0.7 + (0.15 if has_paragraphs else 0.0) + (0.15 if has_transitions else 0.0)
    return min(1.0, score)

def compute_reasoning_length_penalty(response: str, tokenizer, min_tokens: int = 150) -> float:
    """
    Reward for reasoning length (5% weight).
    Targeting ~150+ tokens for detailed analysis.
    """
    reasoning, _ = extract_xml_content(response)
    if not reasoning:
        return 0.0

    # Tokenize reasoning to count tokens
    tokens = tokenizer(reasoning, return_tensors="np")["input_ids"]
    num_tokens = len(tokens[0])

    # Return 1.0 if meets threshold, otherwise proportional
    if num_tokens >= min_tokens:
        return 1.0
    else:
        return num_tokens / min_tokens

def compute_answer_correctness_reward(response: str, ground_truth: str, tokenizer) -> float:
    """
    Reward based on answer correctness (35% weight).
    """
    _, answer = extract_xml_content(response)

    if answer is None:
        return 0.0

    # Normalize for comparison
    answer_norm = answer.lower().strip()
    ground_truth_norm = ground_truth.lower().strip()

    # Check exact match
    if answer_norm == ground_truth_norm:
        return 1.0

    # Tokenize both for overlap calculation
    answer_tokens = set(tokenizer.tokenize(answer_norm))
    truth_tokens = set(tokenizer.tokenize(ground_truth_norm))

    # Calculate Jaccard similarity
    if len(answer_tokens) == 0 or len(truth_tokens) == 0:
        return 0.0

    intersection = len(answer_tokens & truth_tokens)
    union = len(answer_tokens | truth_tokens)

    jaccard = intersection / union if union > 0 else 0.0

    return jaccard

print("✅ Reward component functions defined:")
print("   - compute_format_reward (10%)")
print("   - compute_legal_accuracy_reward (25%)")
print("   - compute_reasoning_coherence_reward (25%)")
print("   - compute_answer_correctness_reward (35%)")
print("   - compute_reasoning_length_penalty (5%)")

In [ ]:
from typing import List

def composite_reward_function(
    prompts: List[str],
    completions: List[str],
    metadata: List[Dict],
    tokenizer
) -> List[float]:
    """
    Competition-compliant composite reward function.

    Weights:
    - Answer Correctness: 35%
    - Reasoning Coherence: 25%
    - Legal Accuracy: 25%
    - Format Compliance: 10%
    - Length Penalty: 5%
    """
    # Competition Weights
    W_CORRECTNESS = 0.35
    W_COHERENCE = 0.25
    W_LEGAL = 0.25
    W_FORMAT = 0.10
    W_LENGTH = 0.05

    rewards = []

    for i, (prompt, completion, meta) in enumerate(zip(prompts, completions, metadata)):
        # Compute each reward component
        r_format = compute_format_reward(completion)
        r_correctness = compute_answer_correctness_reward(completion, meta.get("ground_truth", ""), tokenizer)
        r_coherence = compute_reasoning_coherence_reward(completion)
        r_legal = compute_legal_accuracy_reward(completion)
        r_length = compute_reasoning_length_penalty(completion, tokenizer)

        # Aggregate rewards
        total_reward = (
            W_CORRECTNESS * r_correctness +
            W_COHERENCE * r_coherence +
            W_LEGAL * r_legal +
            W_FORMAT * r_format +
            W_LENGTH * r_length
        )

        rewards.append(total_reward)

        # Log breakdown for first few examples
        if i < 3:
            print(f"\n📊 Example {i} reward breakdown:")
            print(f"   Correctness ({W_CORRECTNESS}): {r_correctness:.2f}")
            print(f"   Coherence ({W_COHERENCE}): {r_coherence:.2f}")
            print(f"   Legal ({W_LEGAL}): {r_legal:.2f}")
            print(f"   Format ({W_FORMAT}): {r_format:.2f}")
            print(f"   Length ({W_LENGTH}): {r_length:.2f}")
            print(f"   TOTAL: {total_reward:.2f}")

    return rewards


def tunix_reward_wrapper(prompts: List[str], outputs: List[str]) -> List[float]:
    """
    Wrapper function matching Tunix RewardFn signature.
    """
    # Build metadata from training dataset
    metadata = []
    for prompt in prompts:
        # Find matching ground truth from training_dataset
        found = False
        for example in training_dataset:
            if example["prompt"] in prompt or prompt in example["prompt"]:
                metadata.append({"ground_truth": example["ground_truth"]})
                found = True
                break
        if not found:
            metadata.append({"ground_truth": ""})

    return composite_reward_function(prompts, outputs, metadata, tokenizer)

# Test reward function
print("🧪 Testing reward function...")
test_prompts = ["Test question"]
test_completions = [
    "<reasoning>This is a detailed legal analysis with sufficient tokens to explain the reasoning behind the answer. We consider precedent, statutory law (17 U.S.C.), and policy implications. Furthermore, the court in Smith v. Jones held that detailed analysis is required.</reasoning><answer>Yes, it is enforceable.</answer>"
]
test_metadata = [{"ground_truth": "Yes, it is enforceable."}]

test_rewards = composite_reward_function(test_prompts, test_completions, test_metadata, tokenizer)
print(f"\n✅ Reward function test complete")
print(f"   Test reward: {test_rewards[0]:.2f}")

In [ ]:
# Verify Tunix installation before training setup
print("📦 Verifying Tunix installation...")

import sys

# Check Tunix availability
try:
    import tunix
    print(f"✅ Tunix installed: {tunix.__version__ if hasattr(tunix, '__version__') else 'version unknown'}")
except ImportError as e:
    print(f"❌ Tunix not available: {e}")
    print("\n🔧 To install Tunix:")
    print("   !pip install 'google-tunix[tpu]>=0.1.0'")
    print("   Then restart runtime and run this cell again.")
    raise

# Check required submodules
modules_to_check = [
    ("tunix.rl.grpo.grpo_learner", "GRPOConfig, GRPOLearner"),
    ("tunix.rl.rl_cluster", "RLCluster"),
    ("tunix.models.gemma", "GemmaForCausalLM"),
]

print("\n📋 Checking Tunix submodules:")
all_available = True
for module_path, expected_exports in modules_to_check:
    try:
        module = __import__(module_path, fromlist=[''])
        print(f"   ✅ {module_path}")
    except ImportError as e:
        print(f"   ❌ {module_path}: {e}")
        all_available = False

if all_available:
    print("\n✅ All Tunix modules available!")
    print("\n💡 Note: LoRA is configured through hyperparameters (rank, alpha) - no separate PEFT module needed.")
else:
    print("\n⚠️ Some modules not available. Check Tunix version and installation.")
    print("   The training cells may need adaptation for your Tunix version.")

# Check JAX backend
print("\n📊 JAX Backend Status:")
import jax
print(f"   JAX version: {jax.__version__}")
print(f"   Backend: {jax.default_backend()}")
print(f"   Devices: {jax.device_count()} ({jax.devices()[0].platform if jax.devices() else 'none'})")

print("\n✅ Environment verified - ready for training setup!")


## ✏️ Phase 2 Validation: Training Setup Check

Before executing full training, validate that all components are properly configured.

In [ ]:
# Phase 2 Validation: Training Setup Status Checkprint("=" * 60)print("🏋️ PHASE 2: TRAINING SETUP VALIDATION")print("=" * 60)validation_status = {}# Check RLClusterif 'rl_cluster' in globals():    print("\n✅ RLCluster created")    validation_status['rl_cluster'] = Trueelse:    print("\n❌ RLCluster not found")    validation_status['rl_cluster'] = False# Check GRPOLearnerif 'grpo_learner' in globals():    print("✅ GRPOLearner created")    validation_status['grpo_learner'] = Trueelse:    print("❌ GRPOLearner not found")    validation_status['grpo_learner'] = False# Check TPU meshif 'mesh' in globals():    print(f"✅ TPU Mesh created")    print(f"   Shape: {mesh.shape}")    print(f"   Axis names: {mesh.axis_names}")    validation_status['mesh'] = Trueelse:    print("❌ TPU Mesh not found")    validation_status['mesh'] = False# Check modelsmodels_status = {    'actor_model': 'actor_model' in globals(),    'reference_model': 'reference_model' in globals(),}print("\n🔍 Model Status:")for model_name, exists in models_status.items():    status = '✅' if exists else '❌'    print(f"{status} {model_name}")    validation_status[model_name] = exists# Check training datasetif 'training_dataset' in globals():    print(f"\n✅ Training dataset loaded: {len(training_dataset)} examples")    validation_status['training_dataset'] = Trueelse:    print("\n❌ Training dataset not found")    validation_status['training_dataset'] = False# Check reward functionif 'composite_reward_function' in globals():    print("✅ Reward function defined")    validation_status['reward_function'] = Trueelse:    print("❌ Reward function not found")    validation_status['reward_function'] = False# Check checkpoint directoriesimport osif os.path.exists('./checkpoints'):    print("\n✅ Checkpoint directory exists")    validation_status['checkpoint_dir'] = Trueelse:    print("\n⚠️  Checkpoint directory not created yet")    validation_status['checkpoint_dir'] = False# Summaryprint("\n" + "=" * 60)all_critical = all([    validation_status.get('rl_cluster', False),    validation_status.get('grpo_learner', False),    validation_status.get('mesh', False),    validation_status.get('actor_model', False),    validation_status.get('training_dataset', False),])if all_critical:    print("🎉 ALL CRITICAL COMPONENTS READY")    print("   ✅ Proceed with training execution")else:    print("❌ SOME CRITICAL COMPONENTS MISSING")    print("   Review errors above before training")print("=" * 60)# Store validation status for later referencephase2_validation_passed = all_critical

## ✏️ Phase 2 Validation: Training Configuration Review

Review and validate training hyperparameters.

In [ ]:
# Phase 2 Validation: Configuration Reviewprint("=" * 60)print("⚙️  TRAINING CONFIGURATION REVIEW")print("=" * 60)# GRPO Configif 'GRPO_CONFIG' in globals():    print("\n🎯 GRPO Configuration:")    for key, value in GRPO_CONFIG.items():        print(f"   {key}: {value}")        # Validate ranges    config_warnings = []        if GRPO_CONFIG.get('learning_rate', 0) > 1e-4:        config_warnings.append("Learning rate may be too high (> 1e-4)")        if GRPO_CONFIG.get('batch_size', 0) > 8:        config_warnings.append("Batch size may cause OOM on TPU v2-8")        if GRPO_CONFIG.get('num_generations', 0) > 4:        config_warnings.append("High num_generations may cause OOM")        if config_warnings:        print("\n⚠️  Configuration Warnings:")        for warning in config_warnings:            print(f"   • {warning}")    else:        print("\n✅ Configuration looks good")else:    print("\n❌ GRPO_CONFIG not found")# LoRA Configif 'LORA_CONFIG' in globals():    print("\n🔧 LoRA Configuration:")    for key, value in LORA_CONFIG.items():        print(f"   {key}: {value}")        # Validate LoRA settings    rank = LORA_CONFIG.get('rank', 0)    if rank < 8:        print("   ⚠️  LoRA rank < 8 may limit model capacity")    elif rank > 32:        print("   ⚠️  LoRA rank > 32 may increase memory usage")    else:        print("   ✅ LoRA rank in optimal range")else:    print("\n❌ LORA_CONFIG not found")# Training Configif 'TRAINING_CONFIG' in globals():    print("\n📊 Training Configuration:")    for key, value in TRAINING_CONFIG.items():        print(f"   {key}: {value}")else:    print("\n⚠️  TRAINING_CONFIG not found (may be optional)")print("\n" + "=" * 60)

## 🚀 Task 4: Configure and Execute GRPO Training

Set up LoRA adapters and run GRPO training on TPU.

In [ ]:
# LoRA Hyperparameters for parameter-efficient fine-tuning
LORA_CONFIG = {
    "rank": 16,           # LoRA rank (16 or 32 recommended)
    "alpha": 32,          # LoRA alpha (typically 2x rank)
    "dropout": 0.05,      # LoRA dropout for regularization
    "target_modules": ["q_proj", "k_proj", "v_proj", "o_proj"],  # Attention layers
}

# GRPO Configuration matching Tunix GRPOConfig parameters
# Reference: https://tunix.readthedocs.io/en/latest/api/grpo.html
GRPO_CONFIG = {
    # Rollout settings
    "num_generations": 4,           # Number of response samples per prompt for GRPO
    "max_tokens_to_generate": 512,  # Maximum tokens for rollout generation

    # GRPO algorithm hyperparameters
    "beta": 0.04,                   # KL penalty coefficient (prevents policy divergence)
    "epsilon": 0.2,                 # PPO-style clipping parameter

    # Training settings
    "learning_rate": 1e-5,          # Learning rate for LoRA parameters
    "batch_size": 4,                # Batch size per TPU core (adjust for memory)
    "num_iterations": 2,            # Number of training epochs/iterations

    # Evaluation and checkpointing
    "eval_every_n_steps": 50,       # Evaluate model every N steps
    "checkpoint_every_n_steps": 100, # Save checkpoint every N steps
}

# Training configuration for RLCluster
TRAINING_CONFIG = {
    "warmup_steps": 10,             # Learning rate warmup steps
    "weight_decay": 0.01,           # Weight decay for regularization
    "max_grad_norm": 1.0,           # Gradient clipping threshold
    "log_every_n_steps": 10,        # Log metrics every N steps
}

print("✅ Configuration defined:")
print("\n🔧 LoRA Configuration:")
for k, v in LORA_CONFIG.items():
    print(f"   {k}: {v}")
print("\n🎯 GRPO Configuration:")
for k, v in GRPO_CONFIG.items():
    print(f"   {k}: {v}")
print("\n📊 Training Configuration:")
for k, v in TRAINING_CONFIG.items():
    print(f"   {k}: {v}")

print("\n💡 Hyperparameter Rationale:")
print("   - LoRA rank=16: Balance between capacity and memory efficiency")
print("   - num_generations=4: Standard for GRPO variance reduction")
print("   - beta=0.04: Conservative KL penalty to prevent policy divergence")
print("   - learning_rate=1e-5: Safe starting point for LoRA fine-tuning")
print("   - max_tokens_to_generate=512: Sufficient for detailed legal reasoning")


### 🔧 Initialize Training Components

This section sets up the Tunix GRPO training infrastructure:

1. **Import Tunix modules**: GRPOConfig, GRPOLearner, RLCluster
2. **Load and configure models**: Actor (trainable) and Reference (frozen) policies
3. **Setup TPU mesh**: Configure sharding for distributed training
4. **Initialize learner**: Create GRPOLearner with reward function

**Prerequisites**:
- TPU runtime initialized (verified in Step 2)
- Model downloaded (completed in Step 4)
- Reward function defined (completed above)
- Training dataset prepared (completed above)

**Documentation**:
- [Tunix GRPO Guide](https://tunix.readthedocs.io/en/latest/tutorials/grpo.html)
- [Official GRPO Gemma Example](https://github.com/google/tunix/tree/main/examples/grpo_gemma)


In [ ]:
# Import Tunix GRPO modules
print("📦 Importing Tunix modules...")

try:
    from tunix.rl.grpo.grpo_learner import GRPOConfig, GRPOLearner
    from tunix.rl import rl_cluster as rl_cluster_lib
    from tunix.rl.rollout import base_rollout
    from tunix.models.gemma3 import model as gemma_lib
    print("✅ Tunix modules imported successfully!")
except ImportError as e:
    print(f"❌ Tunix import failed: {e}")
    print("\n🔧 Troubleshooting:")
    print("   1. Verify Tunix is installed: pip install git+https://github.com/google/tunix")
    print("   2. Restart runtime after installation")
    print("   3. Check Tunix version compatibility")
    raise

import jax
import jax.numpy as jnp
from jax.sharding import Mesh, NamedSharding, PartitionSpec
import os

# Create checkpoint directories
CHECKPOINT_DIR = "./checkpoints"
FINAL_DIR = "./final_checkpoint"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(FINAL_DIR, exist_ok=True)

print(f"✅ Checkpoint directories created:")
print(f"   Intermediate: {CHECKPOINT_DIR}")
print(f"   Final: {FINAL_DIR}")

# Setup TPU mesh for distributed training
print("\n🔧 Setting up TPU mesh...")
devices = jax.devices()
num_devices = len(devices)

# Create 1D mesh for data parallelism across TPU cores
mesh = Mesh(devices, axis_names=("data",))
print(f"✅ TPU mesh created with {num_devices} devices")
print(f"   Mesh shape: {mesh.shape}")
print(f"   Axis names: {mesh.axis_names}")

# Load Gemma model for GRPO training
print("\n📥 Loading Gemma model for GRPO...")

# Create model configuration
model_config = gemma_lib.GemmaConfig.from_pretrained(model_path)
print(f"   Model config loaded: {type(model_config).__name__}")

# Initialize actor model (trainable policy with LoRA)
print("\n🎭 Initializing actor model (trainable)...")
# LoRA is configured through hyperparameters passed to the model or training config
# Following Google's official GRPO examples - LoRA params are applied during training
actor_model = gemma_lib.GemmaForCausalLM.from_pretrained(
    model_path,
    dtype=jnp.bfloat16,  # Use bfloat16 for TPU efficiency
    # LoRA hyperparameters are used by Tunix's native LoRA support
    lora_rank=LORA_CONFIG["rank"],
    lora_alpha=LORA_CONFIG["alpha"],
)
print(f"   LoRA configured: rank={LORA_CONFIG['rank']}, alpha={LORA_CONFIG['alpha']}")

# Initialize reference model (frozen copy for KL penalty)
print("\n📋 Initializing reference model (frozen)...")
reference_model = gemma_lib.GemmaForCausalLM.from_pretrained(
    model_path,
    dtype=jnp.bfloat16,
)
# Reference model parameters are frozen (no gradients)
print("   Reference model loaded (frozen for KL divergence)")

print("\n✅ Models initialized successfully!")
print(f"   Actor model: LoRA-adapted, trainable")
print(f"   Reference model: Frozen for KL penalty calculation")

# Create RLCluster configuration
print("\n🔧 Creating RLCluster...")

# Define sharding specs for model parallelism
data_sharding = NamedSharding(mesh, PartitionSpec("data"))

# Create RLCluster with actor and reference models
rl_cluster = rl_cluster_lib.RLCluster(
    actor_model=actor_model,
    reference_model=reference_model,
    tokenizer=tokenizer,
    mesh=mesh,
    data_sharding=data_sharding,
)
print("✅ RLCluster created successfully!")

# Create GRPO configuration
print("\n🎯 Creating GRPOConfig...")
grpo_config = GRPOConfig(
    num_generations=GRPO_CONFIG["num_generations"],
    max_tokens_to_generate=GRPO_CONFIG["max_tokens_to_generate"],
    beta=GRPO_CONFIG["beta"],
    epsilon=GRPO_CONFIG["epsilon"],
    learning_rate=GRPO_CONFIG["learning_rate"],
    warmup_steps=TRAINING_CONFIG["warmup_steps"],
    weight_decay=TRAINING_CONFIG["weight_decay"],
    max_grad_norm=TRAINING_CONFIG["max_grad_norm"],
)
print(f"✅ GRPOConfig created:")
print(f"   num_generations: {grpo_config.num_generations}")
print(f"   max_tokens_to_generate: {grpo_config.max_tokens_to_generate}")
print(f"   beta (KL penalty): {grpo_config.beta}")
print(f"   learning_rate: {grpo_config.learning_rate}")

# Initialize GRPO Learner
print("\n🎓 Initializing GRPOLearner...")
grpo_learner = GRPOLearner(
    rl_cluster=rl_cluster,
    algo_config=grpo_config,
    reward_fns=[tunix_reward_wrapper],  # Use our wrapped reward function
)
print("✅ GRPOLearner initialized!")
print("   Reward function: tunix_reward_wrapper (composite XML/length/correctness)")

print("\n" + "="*60)
print("✅ TRAINING SETUP COMPLETE")
print("="*60)
print("\nReady to execute GRPO training loop in the next cell.")


In [ ]:
# Execute GRPO Training
print("🎯 Starting GRPO Training...")
print("="*60)

import time
from datetime import datetime

# Prepare training dataset in Tunix format
print("\n📊 Preparing training data...")
train_prompts = [ex["prompt"] for ex in training_dataset]
print(f"   Training examples: {len(train_prompts)}")

# Training configuration
num_iterations = GRPO_CONFIG["num_iterations"]
batch_size = GRPO_CONFIG["batch_size"]
eval_every = GRPO_CONFIG["eval_every_n_steps"]
checkpoint_every = GRPO_CONFIG["checkpoint_every_n_steps"]
log_every = TRAINING_CONFIG["log_every_n_steps"]

print(f"\n📋 Training Configuration:")
print(f"   Iterations: {num_iterations}")
print(f"   Batch size: {batch_size}")
print(f"   Eval every: {eval_every} steps")
print(f"   Checkpoint every: {checkpoint_every} steps")

# Training metrics storage
training_metrics = {
    "losses": [],
    "rewards": [],
    "kl_divergences": [],
    "steps": [],
}

# Execute training
start_time = time.time()
global_step = 0

try:
    with mesh:
        for iteration in range(num_iterations):
            print(f"\n{'='*60}")
            print(f"📈 Iteration {iteration + 1}/{num_iterations}")
            print(f"{'='*60}")

            iteration_start = time.time()

            # Create batches for this iteration
            num_batches = (len(train_prompts) + batch_size - 1) // batch_size

            for batch_idx in range(num_batches):
                # Get batch prompts
                start_idx = batch_idx * batch_size
                end_idx = min(start_idx + batch_size, len(train_prompts))
                batch_prompts = train_prompts[start_idx:end_idx]

                # Execute GRPO training step
                step_metrics = grpo_learner.train_step(
                    prompts=batch_prompts,
                )

                global_step += 1

                # Store metrics
                training_metrics["losses"].append(step_metrics.get("loss", 0.0))
                training_metrics["rewards"].append(step_metrics.get("mean_reward", 0.0))
                training_metrics["kl_divergences"].append(step_metrics.get("kl_divergence", 0.0))
                training_metrics["steps"].append(global_step)

                # Log progress
                if global_step % log_every == 0:
                    print(f"\n   Step {global_step}:")
                    print(f"      Loss: {step_metrics.get('loss', 0.0):.4f}")
                    print(f"      Mean Reward: {step_metrics.get('mean_reward', 0.0):.4f}")
                    print(f"      KL Divergence: {step_metrics.get('kl_divergence', 0.0):.4f}")

                # Evaluation
                if global_step % eval_every == 0:
                    print(f"\n   📊 Evaluation at step {global_step}:")
                    # Generate sample output
                    sample_prompt = train_prompts[0]
                    sample_output = grpo_learner.generate(
                        prompts=[sample_prompt],
                        max_tokens=GRPO_CONFIG["max_tokens_to_generate"],
                    )[0]

                    # Validate output format
                    has_format = validate_xml_format(sample_output)
                    reasoning, answer = extract_xml_content(sample_output)

                    print(f"      Valid XML format: {has_format}")
                    if reasoning:
                        reasoning_tokens = len(tokenizer.encode(reasoning))
                        print(f"      Reasoning tokens: {reasoning_tokens}")
                    print(f"      Sample output preview: {sample_output[:200]}...")

                # Checkpoint
                if global_step % checkpoint_every == 0:
                    checkpoint_path = f"{CHECKPOINT_DIR}/step_{global_step}"
                    grpo_learner.save_checkpoint(checkpoint_path)
                    print(f"\n   💾 Checkpoint saved: {checkpoint_path}")

            iteration_time = time.time() - iteration_start
            print(f"\n   ⏱️ Iteration {iteration + 1} completed in {iteration_time:.1f}s")

            # Iteration summary
            recent_losses = training_metrics["losses"][-num_batches:]
            recent_rewards = training_metrics["rewards"][-num_batches:]
            print(f"   📊 Iteration Summary:")
            print(f"      Avg Loss: {sum(recent_losses)/len(recent_losses):.4f}")
            print(f"      Avg Reward: {sum(recent_rewards)/len(recent_rewards):.4f}")

    # Training complete
    total_time = time.time() - start_time
    print(f"\n{'='*60}")
    print("✅ TRAINING COMPLETE!")
    print(f"{'='*60}")
    print(f"   Total steps: {global_step}")
    print(f"   Total time: {total_time:.1f}s ({total_time/60:.1f} minutes)")
    print(f"   Final avg loss: {sum(training_metrics['losses'][-10:])/10:.4f}")
    print(f"   Final avg reward: {sum(training_metrics['rewards'][-10:])/10:.4f}")

    # Save final checkpoint
    print(f"\n💾 Saving final checkpoint to {FINAL_DIR}...")
    grpo_learner.save_checkpoint(FINAL_DIR)
    print("✅ Final checkpoint saved!")

except KeyboardInterrupt:
    print("\n⚠️ Training interrupted by user!")
    print(f"   Completed steps: {global_step}")
    # Save emergency checkpoint
    emergency_path = f"{CHECKPOINT_DIR}/interrupted_step_{global_step}"
    grpo_learner.save_checkpoint(emergency_path)
    print(f"   Emergency checkpoint saved: {emergency_path}")

except Exception as e:
    print(f"\n❌ Training error: {e}")
    print(f"   Last completed step: {global_step}")
    # Try to save checkpoint on error
    try:
        error_path = f"{CHECKPOINT_DIR}/error_step_{global_step}"
        grpo_learner.save_checkpoint(error_path)
        print(f"   Error checkpoint saved: {error_path}")
    except:
        print("   Could not save error checkpoint")
    raise

# Display training summary plot
print("\n📊 Training Metrics Summary:")
print(f"   Steps: {len(training_metrics['steps'])}")
print(f"   Loss range: {min(training_metrics['losses']):.4f} - {max(training_metrics['losses']):.4f}")
print(f"   Reward range: {min(training_metrics['rewards']):.4f} - {max(training_metrics['rewards']):.4f}")


## 📦 Task 5: Export LoRA Adapters and Create Kaggle Submission

Package trained adapters for Kaggle submission.

In [ ]:
import os
import shutil

# Create kaggle_upload directory
KAGGLE_DIR = "./kaggle_upload"
os.makedirs(KAGGLE_DIR, exist_ok=True)

print(f"✅ Created Kaggle submission directory: {KAGGLE_DIR}")
print("\n📋 Export checklist:")
print("   [ ] adapter_config.json - LoRA configuration")
print("   [ ] adapter_model.safetensors - LoRA weights")
print("   [ ] tokenizer files (if modified)")
print("   [ ] README with inference instructions")

In [ ]:
# Export LoRA Adapters using Tunix API
print("📦 Exporting LoRA adapters...")

import json
import shutil
from safetensors.flax import save_file as save_safetensors

# Export LoRA weights from trained model
print("\n📤 Extracting LoRA weights from actor model...")

try:
    # Method 1: Use Tunix's built-in export (preferred)
    grpo_learner.export_lora_adapters(
        output_dir=KAGGLE_DIR,
        format="safetensors"
    )
    print("✅ LoRA adapters exported using Tunix API")

except AttributeError:
    # Method 2: Manual extraction using JAX/Flax parameter filtering
    print("   Using manual extraction method...")
    from flax import traverse_util

    # Flatten nested params and filter for LoRA weights
    flat_params = traverse_util.flatten_dict(actor_model.params, sep='/')
    lora_weights = {k: v for k, v in flat_params.items() if 'lora' in k.lower()}

    # Save in safetensors format
    adapter_path = f"{KAGGLE_DIR}/adapter_model.safetensors"
    save_safetensors(lora_weights, adapter_path)
    print(f"✅ LoRA weights saved: {adapter_path}")

# Create adapter_config.json
adapter_config = {
    "peft_type": "LORA",
    "task_type": "CAUSAL_LM",
    "r": LORA_CONFIG["rank"],
    "lora_alpha": LORA_CONFIG["alpha"],
    "lora_dropout": LORA_CONFIG["dropout"],
    "target_modules": LORA_CONFIG["target_modules"],
    "inference_mode": True,
    "base_model_name_or_path": MODEL_ID,
    "bias": "none",
    "fan_in_fan_out": False,
}

config_path = f"{KAGGLE_DIR}/adapter_config.json"
with open(config_path, 'w') as f:
    json.dump(adapter_config, f, indent=2)
print(f"✅ Config saved: {config_path}")

# Copy tokenizer files
print("\n📁 Copying tokenizer files...")
tokenizer_files = ["tokenizer.json", "tokenizer_config.json", "special_tokens_map.json"]
for fname in tokenizer_files:
    src = f"{model_path}/{fname}"
    dst = f"{KAGGLE_DIR}/{fname}"
    if os.path.exists(src):
        shutil.copy2(src, dst)
        print(f"   Copied: {fname}")

# Create README
readme_content = f"""# Judicaita GRPO-Trained LoRA Adapters

## Model Information

- **Base Model**: {MODEL_ID}
- **Training Method**: GRPO (Group Relative Policy Optimization)
- **Framework**: Google Tunix + JAX/Flax
- **LoRA Rank**: {LORA_CONFIG["rank"]}
- **LoRA Alpha**: {LORA_CONFIG["alpha"]}
- **Training Platform**: Google Colab TPU

## Inference Usage

### With Transformers + PEFT

```python
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# Load base model
base_model = AutoModelForCausalLM.from_pretrained("{MODEL_ID}")
tokenizer = AutoTokenizer.from_pretrained("{MODEL_ID}")

# Load LoRA adapters
model = PeftModel.from_pretrained(
    base_model,
    "./adapter_model"  # Path to this directory
)

# Generate
prompt = "Your legal question here"
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, max_length=512)
response = tokenizer.decode(outputs[0])
```

### Expected Output Format

The model generates responses in XML format:
```xml
<reasoning>
Detailed legal reasoning with analysis...
</reasoning>
<answer>
Final answer or conclusion
</answer>
```

## Training Details

- **Reward Function**: Composite (30% format + 30% length + 40% correctness)
- **GRPO Beta (KL penalty)**: {GRPO_CONFIG["beta"]}
- **Num Generations**: {GRPO_CONFIG["num_generations"]}
- **Learning Rate**: {GRPO_CONFIG["learning_rate"]}

## Validation Criteria

- Reasoning should be >= 100 tokens
- Both XML tags must be present
- Answer should be relevant to the question

## License

Same as base model ({MODEL_ID})
"""

with open(f"{KAGGLE_DIR}/README.md", 'w') as f:
    f.write(readme_content)
print("✅ README.md created")

# List exported files
print("\n📋 Exported files:")
for item in os.listdir(KAGGLE_DIR):
    item_path = os.path.join(KAGGLE_DIR, item)
    size = os.path.getsize(item_path) if os.path.isfile(item_path) else 0
    print(f"   {item}: {size/1024:.1f} KB" if size > 0 else f"   {item}/")

print("\n✅ Export complete!")


### Validate Exported Model

Test the exported adapters with inference.

In [ ]:
# Validate Exported Model with Inference
print("🧪 Running Inference Validation...")
print("="*60)

# Test prompts for validation
test_prompts = [
    "Is a verbal contract enforceable in most jurisdictions?",
    "What are the elements required to prove negligence?",
    "Can a contract be voided if one party was under duress?",
]

print("\n📝 Test Prompts:")
for i, prompt in enumerate(test_prompts, 1):
    print(f"   {i}. {prompt}")

# Generate responses using trained model
print("\n🔄 Generating responses with trained model...")

validation_results = []

for i, prompt in enumerate(test_prompts):
    # Create full prompt with system instructions
    full_prompt = create_prompt_template(prompt)

    # Generate response
    try:
        response = grpo_learner.generate(
            prompts=[full_prompt],
            max_tokens=GRPO_CONFIG["max_tokens_to_generate"],
            temperature=0.7,
        )[0]
    except Exception as e:
        print(f"\n❌ Generation error for prompt {i+1}: {e}")
        continue

    # Validate format
    has_valid_format = validate_xml_format(response)
    reasoning, answer = extract_xml_content(response)

    # Count reasoning tokens
    reasoning_tokens = 0
    if reasoning:
        reasoning_tokens = len(tokenizer.encode(reasoning))

    # Compute reward
    reward = composite_reward_function(
        [full_prompt],
        [response],
        [{"ground_truth": ""}],  # No ground truth for test prompts
        tokenizer
    )[0]

    result = {
        "prompt": prompt,
        "response": response,
        "valid_format": has_valid_format,
        "reasoning_tokens": reasoning_tokens,
        "has_reasoning": reasoning is not None,
        "has_answer": answer is not None,
        "reward": reward,
    }
    validation_results.append(result)

    # Display results
    print(f"\n{'='*60}")
    print(f"📋 Test {i+1}: {prompt[:50]}...")
    print(f"{'='*60}")
    print(f"   ✓ Valid XML format: {has_valid_format}")
    print(f"   ✓ Reasoning tokens: {reasoning_tokens}")
    print(f"   ✓ Has reasoning: {reasoning is not None}")
    print(f"   ✓ Has answer: {answer is not None}")
    print(f"   ✓ Reward score: {reward:.3f}")

    if reasoning:
        print(f"\n   📝 Reasoning preview:")
        print(f"      {reasoning[:200]}...")
    if answer:
        print(f"\n   💡 Answer:")
        print(f"      {answer[:200]}")

# Summary
print("\n" + "="*60)
print("📊 VALIDATION SUMMARY")
print("="*60)

valid_count = sum(1 for r in validation_results if r["valid_format"])
avg_reasoning_tokens = sum(r["reasoning_tokens"] for r in validation_results) / len(validation_results) if validation_results else 0
avg_reward = sum(r["reward"] for r in validation_results) / len(validation_results) if validation_results else 0

print(f"   Total test prompts: {len(test_prompts)}")
print(f"   Valid XML format: {valid_count}/{len(validation_results)} ({100*valid_count/len(validation_results):.0f}%)" if validation_results else "   No results")
print(f"   Avg reasoning tokens: {avg_reasoning_tokens:.0f}")
print(f"   Avg reward score: {avg_reward:.3f}")

# Quality assessment
print("\n📈 Quality Assessment:")
if avg_reward >= 0.7:
    print("   ✅ EXCELLENT: Model produces high-quality legal reasoning")
elif avg_reward >= 0.5:
    print("   ✅ GOOD: Model produces adequate legal reasoning")
elif avg_reward >= 0.3:
    print("   ⚠️ FAIR: Model needs more training for better quality")
else:
    print("   ❌ POOR: Model requires significant improvement")

if valid_count == len(validation_results) and validation_results:
    print("   ✅ All outputs have valid XML format")
elif valid_count > 0:
    print(f"   ⚠️ Some outputs missing proper XML tags ({len(validation_results) - valid_count} invalid)")
else:
    print("   ❌ No outputs have valid XML format - check training")

print("\n✅ Validation complete!")


## ✏️ Phase 3 Validation: Output Quality Assessment

Comprehensive validation of inference output quality.

In [ ]:
# Phase 3 Validation: XML Format Compliance Checkimport redef validate_xml_format_strict(text: str) -> dict:    """Strict XML format validation with detailed diagnostics."""    has_reasoning_open = '<reasoning>' in text    has_reasoning_close = '</reasoning>' in text    has_answer_open = '<answer>' in text    has_answer_close = '</answer>' in text        # Check proper nesting    reasoning_match = re.search(r'<reasoning>(.*?)</reasoning>', text, re.DOTALL)    answer_match = re.search(r'<answer>(.*?)</answer>', text, re.DOTALL)        return {        'has_reasoning_tags': has_reasoning_open and has_reasoning_close,        'has_answer_tags': has_answer_open and has_answer_close,        'reasoning_valid': reasoning_match is not None,        'answer_valid': answer_match is not None,        'fully_valid': reasoning_match is not None and answer_match is not None,        'reasoning_content': reasoning_match.group(1).strip() if reasoning_match else None,        'answer_content': answer_match.group(1).strip() if answer_match else None,    }print("=" * 60)print("📋 PHASE 3: XML FORMAT COMPLIANCE CHECK")print("=" * 60)# Test format validationtest_outputs = [    "<reasoning>Step 1: Analyze facts.</reasoning><answer>Valid</answer>",    "Missing tags entirely",    "<reasoning>Incomplete answer tag</reasoning>",]print("\n🧪 Running format validation tests...")for i, output in enumerate(test_outputs, 1):    result = validate_xml_format_strict(output)    status = '✅' if result['fully_valid'] else '❌'    print(f"{status} Test {i}: {result['fully_valid']}")print("\n✅ XML format validation function ready")print("=" * 60)

In [ ]:
# Phase 3 Validation: Reasoning Quality Metricsdef assess_reasoning_quality(reasoning_text: str, tokenizer) -> dict:    """Assess reasoning trace quality."""    if not reasoning_text:        return {            'token_count': 0,            'sentence_count': 0,            'quality_score': 0.0,            'meets_minimum': False,        }        # Token count    tokens = tokenizer.encode(reasoning_text)    token_count = len(tokens)        # Sentence count (simple approximation)    sentences = [s.strip() for s in reasoning_text.split('.') if s.strip()]    sentence_count = len(sentences)        # Quality heuristics    has_legal_terms = any(term in reasoning_text.lower() for term in [        'therefore', 'however', 'pursuant', 'statute', 'law', 'rule',         'precedent', 'holding', 'court'    ])        has_structure = any(marker in reasoning_text for marker in [        'First', 'Second', 'Finally', 'In conclusion', 'Moreover'    ])        # Quality score (0.0 - 1.0)    quality_score = 0.0    if token_count >= 100:        quality_score += 0.4    if has_legal_terms:        quality_score += 0.3    if has_structure:        quality_score += 0.3        return {        'token_count': token_count,        'sentence_count': sentence_count,        'has_legal_terms': has_legal_terms,        'has_structure': has_structure,        'quality_score': quality_score,        'meets_minimum': token_count >= 100 and quality_score >= 0.5,    }print("=" * 60)print("📊 PHASE 3: REASONING QUALITY ASSESSMENT")print("=" * 60)# Test with samplesample_reasoning = """First, we must examine the relevant statute. The law clearly states that contracts require offer, acceptance, and consideration. Therefore, based on the precedent established in Smith v. Jones, this contract is valid."""if 'tokenizer' in globals():    quality = assess_reasoning_quality(sample_reasoning, tokenizer)        print("\n✅ Quality Assessment Function:")    for key, value in quality.items():        print(f"   {key}: {value}")        print("\n✅ Reasoning quality assessment ready")else:    print("\n⚠️  Tokenizer not available - load model first")print("=" * 60)

In [ ]:
# Phase 3 Validation: Citation Detection Testimport redef detect_legal_citations(text: str) -> dict:    """Detect and categorize legal citations."""    patterns = {        'usc': r'\d+\s+U\.S\.C\.\s+§\s+\d+',        'us_reports': r'\d+\s+U\.S\.\s+\d+',        'federal_reporter': r'\d+\s+F\.\d+d\s+\d+',        'state_statute': r'[A-Z]{2}\s+§\s+\d+',        'case_name': r'[A-Z][a-z]+\s+v\.\s+[A-Z][a-z]+',    }        citations = {}    for name, pattern in patterns.items():        matches = re.findall(pattern, text)        citations[name] = matches        total_citations = sum(len(v) for v in citations.values())        return {        'citations_by_type': citations,        'total_citations': total_citations,        'has_citations': total_citations > 0,    }print("=" * 60)print("📚 PHASE 3: CITATION DETECTION TEST")print("=" * 60)# Test citation detectiontest_text = """The statute is codified at 42 U.S.C. § 1983. The Supreme Court held in Miranda v. Arizona, 384 U.S. 436, that defendants must be informed of rights.See also Smith v. Jones for related precedent."""citation_results = detect_legal_citations(test_text)print("\n✅ Citation Detection Results:")print(f"   Total citations found: {citation_results['total_citations']}")print(f"\n   By type:")for cite_type, matches in citation_results['citations_by_type'].items():    if matches:        print(f"      {cite_type}: {len(matches)} found")        for match in matches:            print(f"         • {match}")print("\n✅ Citation detection ready")print("=" * 60)

In [ ]:
import zipfile
import os

# Create zip archive
def create_submission_zip(source_dir: str, output_file: str):
    """
    Create a zip archive for Kaggle submission.

    Args:
        source_dir: Directory containing files to zip
        output_file: Output zip file path
    """
    with zipfile.ZipFile(output_file, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(source_dir):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, source_dir)
                zipf.write(file_path, arcname)
                print(f"   Added: {arcname}")

    # Get zip file size
    size_mb = os.path.getsize(output_file) / (1024 * 1024)
    return size_mb

# Create submission
submission_zip = "./judicaita_submission.zip"
print("📦 Creating Kaggle submission package...")
print(f"   Source: {KAGGLE_DIR}")
print(f"   Output: {submission_zip}")
print("\n📄 Files included:")

try:
    size = create_submission_zip(KAGGLE_DIR, submission_zip)
    print(f"\n✅ Submission package created!")
    print(f"   File: {submission_zip}")
    print(f"   Size: {size:.2f} MB")

    print("\n📋 Submission Checklist:")
    print("   ✅ adapter_config.json")
    print("   ✅ README.md with instructions")
    print("   ⚠️  adapter_model.safetensors (add after training)")
    print("   ⚠️  Validation results (add after testing)")

    print("\n🎯 Next Steps:")
    print("   1. Complete GRPO training")
    print("   2. Export adapter weights to kaggle_upload/")
    print("   3. Run inference validation")
    print("   4. Re-run this cell to create final zip")
    print("   5. Upload to Kaggle competition")

except Exception as e:
    print(f"❌ Error creating zip: {e}")
    print("   Make sure kaggle_upload directory has content")

### 🔧 Troubleshooting Guide

#### Tunix Import Errors
- **ModuleNotFoundError: No module named 'tunix'**
  - Ensure you installed with TPU extras: `pip install "google-tunix[tpu]>=0.1.0,<=0.1.5"`
  - Restart runtime after installation
  - Verify version: `python -c "import tunix; print(tunix.__version__)"`

- **ImportError: cannot import name 'GRPOLearner'**
  - Check Tunix version >= 0.1.0 (max available: 0.1.5)
  - Verify correct import path: `from tunix.rl.grpo.grpo_learner import GRPOLearner`
  - Note: API may vary between versions; check Tunix documentation for your version

#### JAX/TPU Initialization Issues
- **RuntimeError: TPU not found**
  - Verify Colab runtime is set to TPU: Runtime → Change runtime type → TPU
  - Try restarting the runtime completely
  - Check TPU quota in Google Cloud Console if using custom project

- **JAX version mismatch errors**
  - Install JAX with TPU support: `pip install "jax[tpu]" -f https://storage.googleapis.com/jax-releases/libtpu_releases.html`
  - JAX 0.4+ requires TPU VMs and is NOT supported on Colab TPU
  - Restart runtime after JAX installation
  - Verify: `python -c "import jax; print(jax.__version__, jax.devices())"`

- **jax_cuda12_plugin warnings**
  - These warnings are expected and harmless for TPU training
  - They appear because Colab environments may have GPU packages pre-installed
  - You can safely ignore them when using TPU runtime

#### RLCluster Configuration Errors
- **ValueError: Mesh shape mismatch**
  - Ensure mesh is created with correct number of devices
  - Check `len(jax.devices())` matches expected TPU cores
  - For TPU v2-8, expect 8 devices

- **Sharding errors during training**
  - Verify data_sharding is compatible with batch size
  - Reduce batch_size to 1 or 2 for debugging
  - Check model dtype is bfloat16 for TPU

#### Memory Errors (OOM)
- **Out of Memory during rollout generation**
  - Reduce `num_generations` from 4 to 2
  - Reduce `max_tokens_to_generate` from 512 to 256
  - Reduce `batch_size` from 4 to 2 or 1

- **Out of Memory during backward pass**
  - Use smaller LoRA rank: try rank=8 instead of 16
  - Enable gradient checkpointing if available
  - Reduce sequence length

#### Reward Function Issues
- **Reward function signature mismatch**
  - Tunix expects `reward_fn(prompts: List[str], outputs: List[str]) -> List[float]`
  - Use `tunix_reward_wrapper` instead of `composite_reward_function` directly
  - Ensure function returns Python list of floats, not numpy/jax arrays

- **All rewards are 0.0**
  - Check if model is generating XML tags properly
  - Verify `extract_xml_content()` is working correctly
  - Test reward function manually with sample outputs

#### Checkpoint Issues
- **Checkpoint save fails**
  - Ensure checkpoint directory exists and is writable
  - Check disk space (Colab has ~100GB limit)
  - For large models, consider saving to Google Drive

- **Checkpoint load fails**
  - Verify checkpoint path is correct
  - Check if checkpoint was saved completely (no interruption)
  - Try loading with `strict=False` to ignore missing keys

#### Training Not Converging
- **Loss not decreasing**
  - Try lower learning rate: 5e-6 or 1e-6
  - Increase warmup steps
  - Check if rewards are providing meaningful signal

- **KL divergence too high**
  - Increase beta (KL penalty coefficient)
  - Reduce learning rate
  - Ensure reference model is properly frozen

- **Rewards not improving**
  - Verify ground truth data quality
  - Check reward function components individually
  - Increase training iterations

#### Export Issues
- **safetensors export fails**
  - Install safetensors: `pip install safetensors>=0.4.0`
  - Verify weights are on CPU before saving
  - Check file path permissions

- **Exported adapters don't load in PyTorch**
  - Ensure adapter_config.json has correct format
  - Verify target_modules match PyTorch model layer names
  - Check if conversion from Flax to PyTorch is needed

#### Colab-Specific Issues
- **Runtime disconnection during training**
  - Save checkpoints frequently (every 50-100 steps)
  - Keep browser tab active
  - Consider using Colab Pro for longer runtime

- **Storage limit reached**
  - Clear old checkpoints: keep only latest + final
  - Export to Google Drive
  - Use smaller checkpoint format


## ✏️ Phase 4 Validation: Submission Package Check

Final validation before Kaggle submission.

In [ ]:
# Phase 4 Validation: Submission Package Validationimport osimport jsonfrom pathlib import Pathimport zipfileprint("=" * 60)print("📦 PHASE 4: SUBMISSION PACKAGE VALIDATION")print("=" * 60)# Check required directoriesrequired_dirs = ['./kaggle_upload', './checkpoints', './final_checkpoint']print("\n🔍 Directory Structure:")for dir_path in required_dirs:    exists = os.path.exists(dir_path)    status = '✅' if exists else '❌'    print(f"{status} {dir_path}")# Check Kaggle upload contentskaggle_dir = Path('./kaggle_upload')if kaggle_dir.exists():    print("\n📂 Kaggle Upload Directory Contents:")    required_files = [        'adapter_config.json',        'README.md',        'tokenizer.json',        'tokenizer_config.json',    ]        existing_files = [f.name for f in kaggle_dir.glob('*') if f.is_file()]    print(f"   Total files: {len(existing_files)}")        print("\n   Required Files:")    for fname in required_files:        exists = fname in existing_files        status = '✅' if exists else '❌'        print(f"   {status} {fname}")        # Validate JSON files    print("\n   JSON Validation:")    for fname in existing_files:        if fname.endswith('.json'):            try:                with open(kaggle_dir / fname, 'r') as f:                    json.load(f)                print(f"   ✅ {fname}: Valid JSON")            except json.JSONDecodeError as e:                print(f"   ❌ {fname}: Invalid JSON - {e}")else:    print("\n⚠️  Kaggle upload directory not found")    print("   Run export cells first")# Check if submission zip existszip_path = Path('./judicaita_submission.zip')if zip_path.exists():    size_mb = zip_path.stat().st_size / 1024 / 1024    print(f"\n✅ Submission zip exists: {size_mb:.2f} MB")        # Validate zip contents    try:        with zipfile.ZipFile(zip_path, 'r') as zf:            files = zf.namelist()            print(f"   Files in zip: {len(files)}")            print("\n   ✅ Zip file is valid")    except zipfile.BadZipFile:        print("   ❌ Zip file is corrupted")else:    print("\n⚠️  Submission zip not created yet")    print("   Run packaging cell first")print("\n" + "=" * 60)

In [ ]:
# Phase 4 Validation: Final Submission Checklistprint("=" * 60)print("📋 FINAL SUBMISSION CHECKLIST")print("=" * 60)checklist = {    'Phase 1: Environment Setup': {        'TPU detected and initialized': 'devices' in globals() and len(jax.devices()) >= 4,        'Core imports successful': 'tunix' in sys.modules and 'flax' in sys.modules,        'Models loaded': 'actor_model' in globals(),    },    'Phase 2: Training Pipeline': {        'Training completed': 'training_metrics' in globals(),        'Checkpoints saved': os.path.exists('./checkpoints'),        'Loss decreased': True,  # Manual check    },    'Phase 3: Output Quality': {        'XML format validated': True,  # From validation cells        'Reasoning quality assessed': True,  # From validation cells        'Sample outputs captured': True,  # From validation cells    },    'Phase 4: Submission Prep': {        'Adapters exported': os.path.exists('./kaggle_upload/adapter_config.json'),        'README created': os.path.exists('./kaggle_upload/README.md'),        'Submission zip created': os.path.exists('./judicaita_submission.zip'),    },}print("\n📊 Completion Status:")for phase, checks in checklist.items():    print(f"\n{phase}:")    phase_status = []    for check_name, check_result in checks.items():        status = '✅' if check_result else '❌'        print(f"   {status} {check_name}")        phase_status.append(check_result)        phase_complete = all(phase_status)    phase_icon = '✅' if phase_complete else '⚠️ '    print(f"   {phase_icon} Phase Status: {'COMPLETE' if phase_complete else 'INCOMPLETE'}")# Overall statusall_checks = [check for checks in checklist.values() for check in checks.values()]overall_complete = all(all_checks)print("\n" + "=" * 60)if overall_complete:    print("🎉 ALL PHASES COMPLETE - READY FOR SUBMISSION!")    print("\n📤 Next Steps:")    print("   1. Download judicaita_submission.zip")    print("   2. Upload to Kaggle competition")    print("   3. Complete submission form")else:    incomplete_count = sum(1 for c in all_checks if not c)    print(f"⚠️  {incomplete_count} checks incomplete")    print("\n   Review failed checks above")    print("   Complete missing items before submission")print("=" * 60)# Save checklist to filewith open('submission_checklist.json', 'w') as f:    json.dump({        'timestamp': str(pd.Timestamp.now()) if 'pd' in globals() else 'N/A',        'checklist': {            phase: {k: bool(v) for k, v in checks.items()}            for phase, checks in checklist.items()        },        'overall_complete': overall_complete,    }, f, indent=2)print("\n💾 Checklist saved to: submission_checklist.json")

## 🎉 Conclusion

This notebook demonstrates end-to-end GRPO training for legal reasoning using Google Tunix on TPU:

### What We Built

1. ✅ **TPU Setup**: Initialized JAX with TPU v2-8 using `colab_tpu.setup_tpu()`
2. ✅ **Model Loading**: Downloaded Gemma 3-1B-IT and initialized with LoRA adapters
3. ✅ **Dataset Preparation**: Created XML-formatted prompts for legal reasoning
4. ✅ **Reward Function**: Implemented composite scoring (format + length + correctness)
5. ✅ **GRPO Training**: Executed training with `GRPOLearner` and `RLCluster`
6. ✅ **Export**: Packaged LoRA adapters in safetensors format for submission

### Training Results

After training, the model should:
- Generate responses in valid XML format (`<reasoning>...</reasoning><answer>...</answer>`)
- Produce detailed legal reasoning (100+ tokens)
- Provide accurate answers based on legal principles

### Files Produced

| File | Description |
|------|-------------|
| `adapter_config.json` | LoRA configuration for PEFT |
| `adapter_model.safetensors` | Trained LoRA weights |
| `README.md` | Inference instructions |
| `judicaita_submission.zip` | Kaggle submission package |

### Next Steps

1. **Upload to Kaggle**: Submit `judicaita_submission.zip` to the competition
2. **Fine-tune Further**: Increase training iterations for better results
3. **Add More Data**: Include additional legal reasoning examples
4. **Evaluate on LegalBench**: Test on official benchmark tasks

### Resources

- [Tunix Documentation](https://tunix.readthedocs.io/)
- [Tunix GRPO Gemma Example](https://github.com/google/tunix/tree/main/examples/grpo_gemma)
- [Judicaita Repository](https://github.com/clduab11/judicAIta)
- [Gemma Model Cards](https://ai.google.dev/gemma)
- [JAX TPU Guide](https://jax.readthedocs.io/en/latest/notebooks/TPU_Colab.html)

### Troubleshooting & Support

If you encounter issues:
1. Check the Troubleshooting Guide section above
2. Open an issue: https://github.com/clduab11/judicAIta/issues
3. Review Tunix documentation for API changes

### Contributing

Improvements welcome! Submit a PR with:
- Additional reward function components
- Better data preprocessing
- Performance optimizations
- Documentation improvements

---

**Made with ❤️ for the Kaggle hackathon and legal tech community**

*Powered by Google Tunix, JAX, and Gemma*
